Clean simbli_school_data.json data and save it as a .csv to simbli_school_data.csv 

In [3]:
import json

with open("simbli_school_data.json",'r') as f:
    simbli_data = json.load(f)

In [1]:
def getNumber(text):
    return ''.join([c for c in text if c.isdigit()])

def getAddress(schoolName, text):
    te = text
    if text.strip() == schoolName.strip():
        return ["", "", ""]
    
    text = text[len(schoolName):].strip()

    address = text

    lastCommaPos = text.rfind(",")

    addressEnd = text.rfind(" ", 0, lastCommaPos)
    if (addressEnd != -1):
        if text[addressEnd + 1:lastCommaPos].strip() == "City":
            addressEnd = text.rfind(" ", 0, addressEnd)

        address = address[:addressEnd].strip()
        text = text[addressEnd:].strip()
    else:
        print("Address end not found: " + schoolName)
        return [None, None, None]
    
    city = text
    
    cityEnd = text.find(",")
    if (cityEnd != -1):
        city = city[:cityEnd].strip()
        text = text[cityEnd:].strip()
    else:
        print("City end not found: " + schoolName)
        return [address, None, None]
    
    zipcode = text[4:].strip() # remove state

    return [address, city, zipcode]

import pandas as pd
from urllib.parse import urlparse

# Function to extract domain name from a URL
def extract_domain(url):
    try:
        # Parse the URL to get the domain
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        
        # Remove 'www.' if present to get the root domain
        domain = domain.replace('www.', '')
        
        # Return the domain (e.g., 'example.com')
        return domain
    except Exception as e:
        return None  # In case of any error, return None (e.g., invalid URL)


In [218]:
import csv

filename = "simbli.csv"

with open(filename, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["simbli_id", "school_name", "state", "address", "city", "zipcode", "phone_number", "fax_number", "website"])

    for key, value in simbli_data.items():
        simbli_id = key
        school_name = value[0] if value[0] else ""
        state = value[1] if value[1] else ""

        seperated_address = getAddress(school_name, value[2][0]) if value[2][0] else ["", "", ""]

        address = seperated_address[0]
        city = seperated_address[1]
        zipcode = seperated_address[2]
        
        phone_number = getNumber(value[2][1]) if value[2][1] else ""
        fax_number = getNumber(value[2][2]) if value[2][2] else ""
        website = value[2][3] if value[2][3] else ""

        writer.writerow([simbli_id, school_name, state, address, city, zipcode, phone_number, fax_number, website])

NameError: name 'simbli_data' is not defined

Get relevant NCES columns and reformat

In [239]:
import pandas as pd

nces = pd.read_csv("../nces2324.csv")

columns = ['LEAID', 'LEA_NAME', 'ST', 'MSTREET1', 'MCITY', 'MZIP', 'PHONE', 'WEBSITE']
nces = nces[columns]

nces.columns = ["LEAID", "school_name", "state", "address", "city", "zipcode", "phone_number", "website"]

nces["phone_number"] = nces["phone_number"].apply(getNumber).astype(int)
nces['phone_number'] = nces['phone_number'].astype(str)
nces['zipcode'] = nces['zipcode'].astype(str)


nces['school_name'] = nces['school_name'].str.replace('SD', '', case=False)
nces["school_name"] = nces["school_name"].str.lower().str.replace(" ", "")
nces['school_name'] = nces['school_name'].str.replace('schooldistrict', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('schooldist', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('district', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('publicschools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('publicschool', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('countyschools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('countyschool', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('county', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('schools', '', case=False)
nces['school_name'] = nces['school_name'].str.replace('school', '', case=False)

nces["address"] = nces["address"].str.lower().str.replace(" ", "")
nces["city"] = nces["city"].str.lower().str.replace(" ", "")

nces['website'] = nces['website'].apply(extract_domain)

nces['website'] = nces['website'].replace("NULL", None)



nces

/var/folders/k7/0d0pjdr93h15578mwngc_klc0000gn/T/ipykernel_5184/758747764.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nces = pd.read_csv("../nces2324.csv")


,LEAID,school_name,state,address,city,zipcode,phone_number,website
0,100002,alabamayouthservices,AL,1000industrialschoolroad,mtmeigs,36057,3342153860,dys.alabama.gov
1,100005,albertvillecity,AL,8379ushighway431,albertville,35950,2568911183,albertk12.org
2,100006,marshall,AL,12380ushighway431s,guntersville,35976,2565823171,marshallk12.org
3,100007,hoovercity,AL,2810metropolitanway,hoover,35243,2054391015,hoovercityschools.net
4,100008,madisoncity,AL,211celticdr,madison,35758,2564648370,madisoncity.k12.al.us
...,...,...,...,...,...,...,...,...
19632,6600041,careertechhighacademycharter,GU,pobox7428,agat,96928,6717278324,careetechguam.org
19633,6900002,commonwealthofthenorthernmarianaislands,MP,pobox501370,saipan,96950,6702373061,cnmipss.org
19634,7200030,puertoricodepartmentofeducation,PR,pobox190759,sanjuan,919,7877592000,de.pr.gov
19635,7800002,saintcroix,VI,2133hospitalstreet,saintcroix,820,3407731747,vide.vi


Get simbli data

In [240]:
simbli = pd.read_csv("simbli_school_data.csv")

simbli['zipcode'] = simbli['zipcode'].astype(str).str.replace('.0', '', regex=False)
simbli['phone_number'] = simbli['phone_number'].astype(str).str.replace('.0', '', regex=False)
simbli['fax_number'] = simbli['fax_number'].astype(str).str.replace('.0', '', regex=False)

simbli['school_name'] = simbli['school_name'].str.replace('SD', '', case=False)
simbli["school_name"] = simbli["school_name"].str.lower().str.replace(" ", "")
simbli['school_name'] = simbli['school_name'].str.replace('schooldistrict', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('publicschools', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('publicschool', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('district', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('schooldist', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('countyschools', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('countyschool', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('county', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('schools', '', case=False)
simbli['school_name'] = simbli['school_name'].str.replace('school', '', case=False)

simbli["address"] = simbli["address"].str.lower().str.replace(" ", "")
simbli["city"] = simbli["city"].str.lower().str.replace(" ", "")

simbli = simbli.drop('fax_number', axis=1)

simbli['website'] = simbli['website'].apply(extract_domain)


simbli
# print(len(simbli))

,simbli_id,school_name,state,address,city,zipcode,phone_number,website
0,2,kirksviller-iii,MO,1901ehamiltonstreet,kirksville,63501,6606657774,kirksville.k12.mo.us
1,4,northandrewco.r-vi,MO,NaN,NaN,nan,nan,northandrew.org
2,6,savannahr-iii,MO,408wmarketst,savannah,64485,8163243144,savannahr3.com
3,10,alamance-burlingtonystem,NC,1712vaughnroad,burlington,27217,3365706060,abss.k12.nc.us
4,11,van-farr-i,MO,NaN,NaN,nan,nan,vf.k12.mo.us
...,...,...,...,...,...,...,...,...
2011,36031828,sequoiaunionelementary,CA,23958ave.324lemon,cove,93244,5595642106,sequoiaunion.org
2012,36031829,gmgc,IA,1710wallaceavenuegreen,mountain,50632,6414742254,gmgschools.socs.net
2013,36031830,glenwoodc,IA,103centralstreet,glenwood,51534,7125279034,glenwoodschools.org
2014,36031831,atlanticcommunity,IA,1406sw7thst.,atlantic,50022,nan,atlanticiaschools.org


In [123]:
def calculateSuccessRate(df):
    conflictCount = (df['LEAID'] == -1).sum()  # Count rows where LEAID is -1

    successCount = df['LEAID'].notna().sum() - conflictCount # Count non-null LEAID values
    totalCount = len(df)  # Total rows in merged_df

    # Success rate as percentage
    success_rate = (successCount / totalCount) * 100
    conflict_rate = (conflictCount / totalCount) * 100

    # Print success rate
    print(f"Success Rate: {success_rate:.2f}%")
    print(f"Conflict Rate: {conflict_rate:.2f}%")


In [241]:
merged_simbli = simbli.copy()
merged_simbli['LEAID'] = None

merged_simbli

,simbli_id,school_name,state,address,city,zipcode,phone_number,website,LEAID
0,2,kirksviller-iii,MO,1901ehamiltonstreet,kirksville,63501,6606657774,kirksville.k12.mo.us,None
1,4,northandrewco.r-vi,MO,NaN,NaN,nan,nan,northandrew.org,None
2,6,savannahr-iii,MO,408wmarketst,savannah,64485,8163243144,savannahr3.com,None
3,10,alamance-burlingtonystem,NC,1712vaughnroad,burlington,27217,3365706060,abss.k12.nc.us,None
4,11,van-farr-i,MO,NaN,NaN,nan,nan,vf.k12.mo.us,None
...,...,...,...,...,...,...,...,...,...
2011,36031828,sequoiaunionelementary,CA,23958ave.324lemon,cove,93244,5595642106,sequoiaunion.org,None
2012,36031829,gmgc,IA,1710wallaceavenuegreen,mountain,50632,6414742254,gmgschools.socs.net,None
2013,36031830,glenwoodc,IA,103centralstreet,glenwood,51534,7125279034,glenwoodschools.org,None
2014,36031831,atlanticcommunity,IA,1406sw7thst.,atlantic,50022,nan,atlanticiaschools.org,None


Start merging

In [243]:
# By name

# Make name lowercase and delete spaces

simbli_stardard_name = simbli.copy()
simbli_stardard_name["school_name"] = simbli["school_name"].str.lower().str.replace(" ", "")

# nces_standard_name = nces
nces_standard_name = nces[nces['school_name'].notna()]
nces_standard_name = nces_standard_name.drop_duplicates(subset=['school_name'], keep=False)

merged_df = pd.merge(simbli_stardard_name, nces_standard_name[['LEAID', 'school_name']], on='school_name', how='left')
merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')




merged_df
# print(len(merged_df))

# Iterate over the DataFrame rows using .iterrows()
for i, row in merged_df.iterrows():
    # Check if both LEAID values are not NaN and do not match
    if (pd.notna(row["LEAID"]) and pd.notna(merged_simbli.loc[i, "LEAID"]) and row["LEAID"] != merged_simbli.loc[i, "LEAID"]) or merged_simbli.loc[i, "LEAID"] == -1:
        # Set both values to NaN
        merged_df.loc[i, "LEAID"] = None
        merged_simbli.loc[i, "LEAID"] = -1

merged_simbli.update(merged_df)
calculateSuccessRate(merged_simbli)
print(len(merged_simbli))


Success Rate: 53.62%
Conflict Rate: 0.00%
2016


In [244]:
# By phone number

simbli_pnumber = simbli

nces_pnumber = nces[nces['phone_number'].notna()]
nces_standard_name = nces_standard_name.drop_duplicates(subset=['phone_number'], keep=False)

merged_df = pd.merge(simbli_pnumber, nces_standard_name[['LEAID', 'phone_number']], on='phone_number', how='left')
merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')


# Iterate over the DataFrame rows using .iterrows()
for i, row in merged_df.iterrows():
    # Check if both LEAID values are not NaN and do not match
    if (pd.notna(row["LEAID"]) and pd.notna(merged_simbli.loc[i, "LEAID"]) and row["LEAID"] != merged_simbli.loc[i, "LEAID"]) or merged_simbli.loc[i, "LEAID"] == -1:
        # Set both values to NaN
        merged_df.loc[i, "LEAID"] = None
        merged_simbli.loc[i, "LEAID"] = -1


merged_simbli.update(merged_df)
calculateSuccessRate(merged_simbli)
print(len(merged_simbli))


Success Rate: 68.01%
Conflict Rate: 0.79%
2016


In [245]:
# By website

simbli_website = simbli[simbli['website'].notna()]

nces_pnumber = nces[nces['website'].notna()]
nces_standard_name = nces_standard_name.drop_duplicates(subset=['website'], keep=False)

merged_df = pd.merge(simbli_website, nces_standard_name[['LEAID', 'website']], on='website', how='left')
merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')


# Iterate over the DataFrame rows using .iterrows()
for i, row in merged_df.iterrows():
    # Check if both LEAID values are not NaN and do not match
    if (pd.notna(row["LEAID"]) and pd.notna(merged_simbli.loc[i, "LEAID"]) and row["LEAID"] != merged_simbli.loc[i, "LEAID"]) or merged_simbli.loc[i, "LEAID"] == -1:
        # Set both values to NaN
        merged_df.loc[i, "LEAID"] = None
        merged_simbli.loc[i, "LEAID"] = -1


merged_simbli.update(merged_df)
calculateSuccessRate(merged_simbli)
print(len(merged_simbli))


Success Rate: 44.44%
Conflict Rate: 36.66%
2016


In [246]:
# By address

# Make name lowercase and delete spaces

simbli_stardard_name = simbli
simbli_stardard_name["address"] = simbli["address"].str.lower().str.replace(" ", "")

nces_standard_name = nces[nces['address'].notna()]
nces_standard_name["school_naddressame"] = nces_standard_name["address"].str.lower().str.replace(" ", "")
nces_standard_name = nces_standard_name.drop_duplicates(subset=['address'], keep=False)

merged_df = pd.merge(simbli_stardard_name, nces_standard_name[['LEAID', 'address']], on='address', how='left')
merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')

# Iterate over the DataFrame rows using .iterrows()
for i, row in merged_df.iterrows():
    # Check if both LEAID values are not NaN and do not match
    if (pd.notna(row["LEAID"]) and pd.notna(merged_simbli.loc[i, "LEAID"]) and row["LEAID"] != merged_simbli.loc[i, "LEAID"]) or merged_simbli.loc[i, "LEAID"] == -1:
        # Set both values to NaN
        merged_df.loc[i, "LEAID"] = None
        merged_simbli.loc[i, "LEAID"] = -1


merged_simbli.update(merged_df)
merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')
calculateSuccessRate(merged_simbli)
print(len(merged_simbli))


Success Rate: 44.59%
Conflict Rate: 37.95%
2016


In [247]:
print(merged_simbli)

      simbli_id               school_name state                  address  \
0             2           kirksviller-iii    MO      1901ehamiltonstreet   
1             4        northandrewco.r-vi    MO                      NaN   
2             6             savannahr-iii    MO             408wmarketst   
3            10  alamance-burlingtonystem    NC           1712vaughnroad   
4            11                van-farr-i    MO                      NaN   
...         ...                       ...   ...                      ...   
2011   36031828    sequoiaunionelementary    CA        23958ave.324lemon   
2012   36031829                      gmgc    IA   1710wallaceavenuegreen   
2013   36031830                 glenwoodc    IA         103centralstreet   
2014   36031831         atlanticcommunity    IA             1406sw7thst.   
2015   36031832              gardengroveu    CA  10331stanfordave.garden   

            city zipcode phone_number                website      LEAID  
0     kirksvi

In [248]:
# State and phone number

import pandas as pd

def matchBy(col1, col2):

    # Filter the DataFrames based on non-null values for state and phone_number (ignoring website)
    simbli_filtered = simbli[simbli[col1].notna() & simbli[col2].notna()]
    nces_filtered = nces[nces[col1].notna() & nces[col2].notna()]

    # Drop duplicates based on 'state' and 'phone_number' from nces
    nces_filtered = nces_filtered.drop_duplicates(subset=[col1, col2], keep=False)

    # Perform merge based on state and phone_number
    merged_df = pd.merge(simbli_filtered, nces_filtered[['LEAID', col1, col2]], 
                        on=[col1, col2], how='left')

    # Ensure LEAID is of the correct type (Int64)
    merged_df['LEAID'] = merged_df['LEAID'].astype('Int64')

    # Iterate over the rows of merged_df and perform operations if needed
    for i, row in merged_df.iterrows():
        # Example: check if LEAID is missing in the merged data and set it to None or handle it
        if pd.isna(row['LEAID']):
            merged_df.at[i, 'LEAID'] = None  # Or handle as needed

    # Update merged_simbli DataFrame with the merged_df
    merged_simbli.update(merged_df)

    merged_simbli.update(merged_df)
    calculateSuccessRate(merged_simbli)
    print(len(merged_simbli))


In [249]:
merged_simbli_backup = merged_simbli.copy()

In [252]:
merge_simbli = merged_simbli_backup.copy()

calculateSuccessRate(merge_simbli)

matchBy('state', 'phone_number')
matchBy('state', 'school_name')
matchBy('state', 'address')
matchBy('zipcode', 'phone_number')
matchBy('zipcode', 'school_name')
matchBy('zipcode', 'address')
matchBy('state', 'website')
matchBy('school_name', 'website')
matchBy('phone_number', 'website')
matchBy('zipcode', 'website')
matchBy('state', 'school_name')
matchBy('school_name', 'phone_number')
matchBy('phone_number', 'city')
matchBy('website', 'city')
matchBy('address', 'city')
matchBy('school_name', 'state')


print(merged_simbli)

Success Rate: 44.59%
Conflict Rate: 37.95%
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
Success Rate: 92.51%
Conflict Rate: 0.05%
2016
      simbli_id               school_name state                  address  \
0             2           kirksviller-iii    MO      1901ehamiltonstreet   
1             4        northandrewco.r-vi    MO      

In [180]:
import pandas as pd

def get_sorted_duplicate_rows(df, subset_columns):
    """Return sorted rows where duplicates exist based on the given subset of columns, ignoring NaN."""
    # Drop rows where any of the subset_columns contain NaN values
    df_clean = df.dropna(subset=subset_columns)
    
    # Find duplicates based on the cleaned DataFrame and sort them
    duplicates = df_clean[df_clean.duplicated(subset=subset_columns, keep=False)]
    
    # Return sorted duplicates
    return duplicates.sort_values(by=subset_columns)

# Define column groups to check for duplicates
column_groups = [
    ["school_name"],
    ["address"],
    ["phone_number"],
    ["website"],
    ["school_name", "address"],
    ["school_name", "phone_number"],
    ["school_name", "website"],
    ["address", "phone_number"]
]

# Create a dictionary to store all duplicate DataFrames
duplicate_dfs = {}

# Generate sorted duplicate DataFrames for both simbli and nces
for col_set in column_groups:
    key = "_".join(col_set)  # Create a key name based on column names
    simbli_duplicates = get_sorted_duplicate_rows(simbli, col_set)
    nces_duplicates = get_sorted_duplicate_rows(nces, col_set)

    # Save both DataFrames as CSVs
    simbli_duplicates.to_csv(f"simbli_duplicates_{key}.csv", index=False)
    nces_duplicates.to_csv(f"nces_duplicates_{key}.csv", index=False)

    # Store the DataFrames (optional, for further reference in code)
    duplicate_dfs[f"simbli_duplicates_{key}"] = simbli_duplicates
    duplicate_dfs[f"nces_duplicates_{key}"] = nces_duplicates

print("CSV files have been saved for all duplicates, ignoring NaN.")


CSV files have been saved for all duplicates, ignoring NaN.


In [253]:
# Remove .0 from LEAID by converting it to an integer type
merged_simbli['LEAID'] = merged_simbli['LEAID'].astype('Int64')  # Ensuring that LEAID is Int64 type

# Save merged_simbli as CSV
merged_simbli.to_csv('merged_simbli.csv', index=False)

# Verify the output
print("CSV saved as 'merged_simbli.csv' and LEAID has been converted.")

CSV saved as 'merged_simbli.csv' and LEAID has been converted.


In [ ]:
import pandas as pd

# Load the CSV files
merged_simbli = pd.read_csv('merged_simbli.csv')
nces = pd.read_csv('../nces2324.csv')

# Merge the DataFrames on LEAID and school_name columns (matching between nces and merged_simbli)
result = pd.merge(merged_simbli[['simbli_id', 'LEAID'], nces[['LEAID', 'LEA_NAME']]], 
                  on='LEAID', how='left')

# Select only LEAID, school_name, and simbli_id columns
result = result[['LEAID', 'LEA_NAME', 'simbli_id']]
result['LEAID'] = result['LEAID'].astype('Int64') 

# Save the result as a new CSV
result.to_csv('merged_leaid_school_simbli.csv', index=False)

# # Verify by displaying the first few rows
# print(result.head())


/var/folders/k7/0d0pjdr93h15578mwngc_klc0000gn/T/ipykernel_5184/2151922582.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nces = pd.read_csv('../nces2324.csv')
